// Databricks notebook source exported at Mon, 8 Feb 2016 23:29:54 UTC


#![Wikipedia Logo](http://sameerf-dbc-labs.s3-website-us-west-2.amazonaws.com/data/wikipedia/images/w_logo_for_labs.png)

# Explore English Wikipedia clickstream
### Time to complete: 20 minutes

#### Business Questions:

* Question # 1) What are the top 10 articles requested from Wikipedia?
* Question # 2) Who sent the most traffic to Wikipedia in Feb 2015?
* Question # 3) What were the top 5 trending articles on Twitter in Feb 2015?
* Question # 4) What are the most requested missing pages?
* Question # 5) What does the traffic inflow vs outflow look like for the most requested pages?
* Question # 6) What does the traffic flow pattern look like for the San Francisco article? Create a visualization for this.

#### Technical Accomplishments:

* Learn how to use the Spark CSV Library to read structured files
* Explore the Spark UIs to understand the performance characteristics of your Spark jobs
* Mix SQL and DataFrames queries
* Join 2 DataFrames
* Create a Google visualization to understand the clickstream traffic for the 'San Francisco' article
* Bonus: Explain in DataFrames and SQL



Dataset: http://datahub.io/dataset/wikipedia-clickstream/resource/be85cc68-d1e6-4134-804a-fd36b94dbb82

Lab idea from [Ellery Wulczyn](https://ewulczyn.github.io/Wikipedia_Clickstream_Getting_Started/)

### Getting to know the Data
How large is the data? Let's use `%fs` to find out.

 `%fs` makes it easy to work with the Databricks File System (DBFS). DBFS is a wrapper around Amazon S3 and makes it easier to do common tasks such as mounting S3 buckets, listing directory contents, making new directories, deleting files, copying files, etc. You can learn more about `dbutils` in Databricks Guide.

In [ ]:
%fs ls /databricks-datasets/wikipedia-datasets/data-001/clickstream/raw-uncompressed

 1322171548 bytes means 1.2 GB.

 The file we are exploring in this lab is the February 2015 English Wikipedia Clickstream data. 

According to Wikimedia: 

>"The data contains counts of (referer, resource) pairs extracted from the request logs of English Wikipedia. When a client requests a resource by following a link or performing a search, the URI of the webpage that linked to the resource is included with the request in an HTTP header called the "referer". This data captures 22 million (referer, resource) pairs from a total of 3.2 billion requests collected during the month of February 2015."

### DataFrames
A `sqlContext` object is your entry point for working with structured data (rows and columns) in Spark.

Let's use the `sqlContext` to read a table of the Clickstream data.

##### First import stuff

In [1]:
import org.apache.spark.sql.functions._
import sqlContext.implicits._

In [2]:
// Notice that the sqlContext in Databricks is actually a HiveContext
sqlContext

org.apache.spark.sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@26ae9861

 A `HiveContext` includes additional features like the ability to write queries using the more complete HiveQL parser, access to Hive UDFs, and the ability to read data from Hive tables. In general, you should always aim to use the `HiveContext` over the more limited `sqlContext`.

 First let's load the data into a DataFrame.

 Use the [Spark CSV Library](https://github.com/databricks/spark-csv) to parse the tab separated file:

In [3]:
//Create a DataFrame with the anticipated structure
val clickstreamDF = sqlContext.read.format("com.databricks.spark.csv")
  .option("header", "true")
  .option("delimiter", "\t")
  .option("mode", "PERMISSIVE")
  .option("inferSchema", "true")
  .load("file:///mnt/ephemeral/summitdata/2015_01_clickstream.tsv")

 The `display()` function shows the DataFrame:

In [4]:
clickstreamDF

org.apache.spark.sql.DataFrame = [prev_id: int, curr_id: int, n: int, prev_title: string, curr_title: string, type: string]

 `printSchema()` prints out the schema, the data types and whether a column can be null:

In [5]:
clickstreamDF.printSchema()

root
 |-- prev_id: integer (nullable = true)
 |-- curr_id: integer (nullable = true)
 |-- n: integer (nullable = true)
 |-- prev_title: string (nullable = true)
 |-- curr_title: string (nullable = true)
 |-- type: string (nullable = true)



 The two id columns (prev_id and curr_id) are not used in this lab, so let's create a new DataFrame without them:

In [6]:
val clickstreamDF2 = clickstreamDF.select($"prev_title", $"curr_title", $"n", $"type")

 `.show()` is similar to the `display()` fucntion, but it shows the data in an ASCII format:

In [7]:
clickstreamDF2.show(5)

+---------------+----------+----+----+
|     prev_title|curr_title|   n|type|
+---------------+----------+----+----+
|    other-empty|        !!| 415|null|
|   other-google|        !!| 113|null|
|other-wikipedia|        !!|  33|null|
|    other-yahoo|       !!!|  25|null|
|   other-google|       !!!|1193|null|
+---------------+----------+----+----+
only showing top 5 rows



 Here is what the 6 columns mean:

- `prev_id`: *(note, we already dropped this)* if the referer does not correspond to an article in the main namespace of English Wikipedia, this value will be empty. Otherwise, it contains the unique MediaWiki page ID of the article corresponding to the referer i.e. the previous article the client was on

- `curr_id`: *(note, we already dropped this)* the MediaWiki unique page ID of the article the client requested

- `prev_title`: the result of mapping the referer URL to the fixed set of values described above

- `curr_title`: the title of the article the client requested

- `n`: the number of occurrences of the (referer, resource) pair

- `type`
  - "link" if the referer and request are both articles and the referer links to the request
  - "redlink" if the referer is an article and links to the request, but the request is not in the production enwiki.page table
  - "other" if the referer and request are both articles but the referer does not link to the request. This can happen when clients search or spoof their refer

 Referers were mapped to a fixed set of values corresponding to internal traffic or external traffic from one of the top 5 global traffic sources to English Wikipedia, based on this scheme:

>- an article in the main namespace of English Wikipedia -> the article title
- any Wikipedia page that is not in the main namespace of English Wikipedia -> `other-wikipedia`
- an empty referer -> `other-empty`
- a page from any other Wikimedia project -> `other-internal`
- Google -> `other-google`
- Yahoo -> `other-yahoo`
- Bing -> `other-bing`
- Facebook -> `other-facebook`
- Twitter -> `other-twitter`
- anything else -> `other-other`

### Reading from disk vs memory

The 1.2 GB Clickstream file is currently on S3, which means each time you scan through it, your Spark cluster has to read the 1.2 GB of data remotely over the network.

 Call the `count()` action to check how many rows are in the DataFrame and to see how long it takes to read the DataFrame from S3:

In [19]:
clickstreamDF2.count()

Long = 21996601

 So it took about 1 minute to read the 1.2 GB file into your Spark cluster. The file has 22.5 million rows/lines. We should cache the DataFrame into memory so it'll be faster to work with:

In [20]:
// cache() is a lazy operation, so we need to call an action (like count) to materialize the cache
clickstreamDF2.cache().count()

WARN  2016-02-13 03:22:44,328 org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


Long = 21996601

 How much faster is the DataFrame to read from memory?

In [21]:
clickstreamDF2.count()

Long = 21996601

 Less than a second!

 
### Question #1:
** What are the top 10 articles requested from Wikipedia?**

 We start by grouping by the current title and summing the number of occurrances of the referrer/resource pair:

In [23]:
clickstreamDF2.groupBy("curr_title").sum().limit(10)

org.apache.spark.sql.DataFrame = [curr_title: string, sum(n): bigint]

 To see just the top 10 articles requested, we also need to order by the sum of n column, in descending order.

** Challenge 1:** Can you build upon the code in the cell above to also order by the sum column in descending order, then limit the results to the top ten?

In [24]:
//Type in your answer here...
clickstreamDF2.groupBy("curr_title").sum().orderBy($"sum(n)".desc).limit(10)

org.apache.spark.sql.DataFrame = [curr_title: string, sum(n): bigint]

 Spark SQL lets you seemlessly move between DataFrames and SQL. We can run the same query using SQL:

In [25]:
//First register the table, so we can call it from SQL
clickstreamDF2.registerTempTable("clickstream")

 First do a simple "Select all" query from the `clickstream` table to make sure it's working:

In [27]:
sqlContext.sql("SELECT * FROM clickstream LIMIT 5").show

+---------------+----------+----+----+
|     prev_title|curr_title|   n|type|
+---------------+----------+----+----+
|    other-empty|        !!| 415|null|
|   other-google|        !!| 113|null|
|other-wikipedia|        !!|  33|null|
|    other-yahoo|       !!!|  25|null|
|   other-google|       !!!|1193|null|
+---------------+----------+----+----+



 Now we can translate our DataFrames query to SQL:

In [32]:
sqlContext.sql("SELECT curr_title, SUM(n) AS top_articles FROM clickstream GROUP BY curr_title ORDER BY top_articles DESC LIMIT 10").show

+--------------------+------------+
|          curr_title|top_articles|
+--------------------+------------+
|           Main_Page|   489603866|
|          Chris_Kyle|     4211238|
|             Malware|     4067814|
|       Charlie_Hebdo|     2581856|
|     Leptin_receptor|     2565856|
|              Chrome|     1792151|
|       Script_kiddie|     1779860|
|American_Sniper_(...|     1753218|
|Winston-Salem/For...|     1542559|
|        Flow_control|     1369143|
+--------------------+------------+



 The most requested articles tend to be about media that was popular in February 2015, with a few exceptions.

 SQL also has some handy commands like `DESC` (describe) to see the schema + data types for the table:

In [29]:
sqlContext.sql("DESC clickstream").show

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|prev_title|   string|       |
|curr_title|   string|       |
|         n|      int|       |
|      type|   string|       |
+----------+---------+-------+



 You can use the `SHOW FUNCTIONS` command to see what functions are supported by Spark SQL:

In [34]:
sqlContext.sql("SHOW FUNCTIONS").show(50)

+--------------------+
|            function|
+--------------------+
|                   !|
|                  !=|
|                   %|
|                   &|
|                   *|
|                   +|
|                   -|
|                   /|
|                   <|
|                  <=|
|                 <=>|
|                  <>|
|                   =|
|                  ==|
|                   >|
|                  >=|
|                   ^|
|                 abs|
|                acos|
|          add_months|
|                 and|
|approx_count_dist...|
|               array|
|      array_contains|
|               ascii|
|                asin|
|         assert_true|
|                atan|
|               atan2|
|                 avg|
|              base64|
|             between|
|              bigint|
|                 bin|
|              binary|
|             boolean|
|                case|
|                cbrt|
|                ceil|
|             ceiling|
|          

 `EXPLAIN` can be used to understand the Physical Plan of the SQL statement:

In [ ]:
%sql EXPLAIN SELECT curr_title, SUM(n) AS top_articles FROM clickstream GROUP BY curr_title ORDER BY top_articles DESC;

 Since Spark SQL is not designed to be a low-latency transactional database (like MySQL or Cassandra), INSERTs, UPDATEs and DELETEs are not supported. (Spark SQL is typically used for batch analysis of data)

 
### Question #2:
** Who sent the most traffic to Wikipedia in Feb 2015?** So, who were the top referers to Wikipedia?

In [35]:
clickstreamDF2
  .groupBy("prev_title")
  .sum()
  .orderBy($"sum(n)".desc)
  .show(10)

+---------------+----------+
|     prev_title|    sum(n)|
+---------------+----------+
|   other-google|1583172874|
|    other-empty|1039020465|
|other-wikipedia|  99610051|
|          other|  83225546|
|     other-bing|  64539160|
|    other-yahoo|  49074812|
|      Main_Page|  25077303|
|  other-twitter|  22541757|
| other-facebook|   2599787|
|     Chris_Kyle|   1547364|
+---------------+----------+
only showing top 10 rows



 The top referer by a large margin is Google. Next comes refererless traffic (usually clients using HTTPS). The third largest sender of traffic to English Wikipedia are Wikipedia pages that are not in the main namespace (ns = 0) of English Wikipedia. Learn about the Wikipedia namespaces here:
https://en.wikipedia.org/wiki/Wikipedia:Project_namespace

Also, note that Twitter sends 10x more requests to Wikipedia than Facebook.

 
### Question #3:
** What were the top 5 trending articles on Twitter in Feb 2015?**

** Challenge 2:** Can you answer this question using DataFrames?

In [39]:
//Type in your answer here
clickstreamDF2
  .filter("prev_title = 'other-twitter'")
  .groupBy("curr_title")
  .sum()
  .orderBy($"sum(n)".desc)
  .limit(5).show

+----------------+------+
|      curr_title|sum(n)|
+----------------+------+
| Andr?_the_Giant|286710|
|Harald_Bluetooth|258130|
|    London_Stone|247418|
|         Yaodong|184078|
|    New_Horizons|174157|
+----------------+------+



 ** Challenge 3:** Try re-writing the query above using SQL:

In [ ]:
%sql SELECT curr_title, SUM(n) AS top_twitter FROM clickstream WHERE prev_title = "other-twitter" GROUP BY curr_title ORDER BY top_twitter DESC LIMIT 5;

 
### Question #4:
** What are the most requested missing pages? ** (These are the articles that someone should create on Wikipedia!)

 The type column of our table has 3 possible values:

In [ ]:
%sql SELECT DISTINCT type FROM clickstream;

 These are described as:
  - **link** - if the referer and request are both articles and the referer links to the request
  - **redlink** - if the referer is an article and links to the request, but the request is not in the production enwiki.page table
  - **other** - if the referer and request are both articles but the referer does not link to the request. This can happen when clients search or spoof their refer

 Redlinks are links to a Wikipedia page that does not exist, either because it has been deleted, or because the author is anticipating the creation of the page. Seeing which redlinks are the most viewed is interesting because it gives some indication about demand for missing content.

Let's find the most popular redlinks:

In [ ]:
display(clickstreamDF2.filter("type = 'redlink'").groupBy("curr_title").sum().orderBy($"sum(n)".desc).limit(5))

 Indeed there doesn't appear to be an article on the Russian actress [Anna Lezhneva](https://en.wikipedia.org/wiki/Anna_Lezhneva) on Wikipedia. Maybe you should create it!

Note that if you clicked on the link for Anna Lezhneva in this cell, then you registered another Redlink for her article.

 
### Question #5:
** What does the traffic inflow vs outflow look like for the most requested pages? **

 Wikipedia users get to their desired article by either searching for the article in a search engine or navigating from one Wikipedia article to another by following a link. For example, depending on which technique a user used to get to his desired article of **San Francisco**, the (`prev_title`, `curr_title`) tuples would look like:
- (`other-google`, `San_Francisco`)
or
- (`Berkeley`, `San_Francisco`)

 Lets look at the ratio of incoming to outgoing links for the most requested pages.

 First, find the pageviews per article:

In [ ]:
val pageviewsPerArticleDF = clickstreamDF2
  .groupBy("curr_title")
  .sum(
  ).withColumnRenamed("sum(n)", "in_count")
  .cache()

pageviewsPerArticleDF.show(10)

 Above we can see that the `.17_Remington` article on Wikipedia in Feb 2015, got 2,143 views.

 Then, find the link clicks per article:

In [ ]:
val linkclicksPerArticleDF = clickstreamDF2
  .groupBy("prev_title")
  .sum()
  .withColumnRenamed("sum(n)", "out_count")
  .cache()

linkclicksPerArticleDF.show(10)

 So, when people went to the `David_Janson` article on Wikipedia in Feb 2015, 340 times they clicked on a link in that article to go to a next article. 

 Join the two DataFrames we just created to get a wholistic picture:

In [ ]:
val in_outDF = pageviewsPerArticleDF.join(linkclicksPerArticleDF, ($"curr_title" === $"prev_title")).orderBy($"in_count".desc)

in_outDF.show(10)

 The `curr_title` and `prev_title` above are the same, so we can just display one of them in the future. Next, add a new `ratio` column to easily see whether there is more `in_count` or `out_count` for an article:

In [ ]:
val in_out_ratioDF = in_outDF.withColumn("ratio", $"out_count" / $"in_count").cache()

in_out_ratioDF.select($"curr_title", $"in_count", $"out_count", $"ratio").show(5)

 We can see above that when clients went to the **Alive** article, almost nobody clicked any links in the article to go on to another article.

But 49% of people who visited the **Fifty Shades of Grey** article clicked on a link in the article and continued to browse Wikipedia.

 
### Question #6:
** What does the traffic flow pattern look like for the "San Francisco" article? Create a visualization for this. **

In [ ]:
in_out_ratioDF.filter("curr_title = 'San_Francisco'").show()

 Hmm, so about 41% of clients who visit the San_Francisco page, click on through to another article.

 Which referrers send the most traffic to the "San Francisco" article?

In [ ]:
%sql SELECT * FROM clickstream WHERE curr_title LIKE 'San_Francisco' ORDER BY n DESC LIMIT 10;

 Here's the same query using DataFrames and `show()`:

In [ ]:
clickstreamDF2.filter($"curr_title".rlike("""^San_Francisco$""")).orderBy($"n".desc).show(10)

 ** Challenge 4:** Which future articles does the San_Francisco article send most traffic onward to? Try writing this query using the DataFrames API:

In [ ]:
//Type in your answer here...
display(clickstreamDF2.filter($"prev_title".rlike("""^San_Francisco$""")).orderBy($"n".desc))

 Above we can see the topics most people are interested in, when they get to the San_Francisco article. The [Golden_Gate_Bridge](https://en.wikipedia.org/wiki/Golden_Gate_Bridge) is the second most clicked on link in the San_Francisco article.

 Finally, we'll use a Google Visualization library to create a Sankey diagram. Sankey diagrams are a flow diagram, in which the width of the arrows are shown proportionally to the flow quantify traffic:

In [12]:
// Note you may need to disable your privacy browser extensions to make this work (especially Privacy Badger)

kernel.magics.html("""
<!DOCTYPE html>
<body>
<script type="text/javascript"
           src="https://www.google.com/jsapi?autoload={'modules':[{'name':'visualization','version':'1.1','packages':['sankey']}]}">
</script>

<div id="sankey_multiple" style="width: 900px; height: 300px;"></div>

<script type="text/javascript">
google.setOnLoadCallback(drawChart);
   function drawChart() {
    var data = new google.visualization.DataTable();
    data.addColumn('string', 'From');
    data.addColumn('string', 'To');
    data.addColumn('number', 'Weight');
    data.addRows([
      ['other-google', 'San_Francisco', 53387],
 ['other-empty', 'San_Francisco', 14299],
 ['other-wikipedia', 'San_Francisco', 7806],
 ['other-other', 'San_Francisco', 2674],
 ['other-bing', 'San_Francisco', 2298],
 ['Main_Page', 'San_Francisco', 1804],
 ['California', 'San_Francisco', 1578],
 ['San_Francisco', 'List_of_people_from_San_Francisco', 1352],
 ['San_Francisco', 'Golden_Gate_Bridge', 1263],
 ['San_Francisco', 'California ', 1064],
 ['San_Francisco', 'Los_Angeles', 1014],
 ['San_Francisco', '1906_San_Francisco_earthquake', 816],
 ['San_Francisco', 'Consolidated_city-county', 804],
 ['San_Francisco', 'Alcatraz_Island', 788],
    ]);
    // Set chart options
    var options = {
      width: 600,
      sankey: {
        link: { color: { fill: '#grey', fillOpacity: 0.3 } },
        node: { color: { fill: '#a61d4c' },
                label: { color: 'black' } },
      }
    };
    // Instantiate and draw our chart, passing in some options.
    var chart = new google.visualization.Sankey(document.getElementById('sankey_multiple'));
    chart.draw(data, options);
   }
</script>
  </body>
</html>""")

<!DOCTYPE html>

 The chart above shows how people get to a Wikipedia article and what articles they click on next.

This diagram shows incoming and outgoing traffic to the "San Francisco" article. We can see that most people found the "San Francisco" page through Google search and only a small fraction of the readers went on to another article (most went on to the "List of people in San Francisco" article)

 Note that it is also possible to programmatically add in the values in the HTML, so you don't have to hand-code it. But to keep things simple, we've hand coded it above.

 
### Bonus:
** Learning about Explain to understand Catalyst internals **

 The `explain()` method can be called on a DataFrame to understand its physical plan:

In [ ]:
in_out_ratioDF.explain()

 You can also pass in `true` to see the logical & physical plans:

In [ ]:
in_out_ratioDF.explain(true)

 This concludes the Clickstream lab.

 
### Homework:
** Recreate the same visualization above, but instead of the "San Francisco" article, choose another topic you care about (maybe `Apache_Spark`?).**

 ###Post lab demos below:

This section will be covered by the instructor using a hands-on demo shortly...

 Which pages multiplied input clicks the most?

In [ ]:
clickstreamDF2.show(3)

In [ ]:
val inClicksDF = clickstreamDF2
  .groupBy($"curr_title")
  .sum()
  .withColumnRenamed("sum(n)", "in_clicks")
  .select($"curr_title", $"in_clicks")
  .as("in")

In [ ]:
inClicksDF.show(5)

In [ ]:
val outClicksDF = clickstreamDF2
	.filter($"type" === "link")
	.groupBy($"prev_title")
	.sum()
	.withColumnRenamed("sum(n)", "out_clicks")
	.select($"prev_title", $"out_clicks")
	.as("out")

In [ ]:
outClicksDF.show(5)

 Notes...Mention that the filter is good, removes 50% of data
Then Aggregate? partial aggregation, dont? worry about details
then shuffle
then reduce side aggregation
Cartesian Product: very expensive? joining every single row on left to rows on right? takes forever
Get?s triggered by UDF.. during join, we?re passing in an arbitrary function.. so the user can do whatever they want in the function?
but take a closer look at the body of the UDF above (title1.toLowercase == title2.toLowerCase, you can see that we?re just doing a simple eqality check
b/c it?s an arbitrary UDF, Catalyst (SQL query optimizer) doesn?t know how to optimize this join b/c it doesn?t understand it.. it?s opaque
Solution: a better UDF, here we explicitly use the built in Spark SQL equality expression that Catalyst does understand. I?m still using a UDF to convert to lowercase, just not for equality
Now verify CP is gone in UI.

In [ ]:
// Define a UDF for comparing article titles
val compareUDF = udf((title1: String, title2: String) => title1.toLowerCase == title2.toLowerCase)

In [ ]:
// Join these 2 DFs to find the (output clicks) / (input clicks) factor
val joinedDF = outClicksDF
	.join(inClicksDF, compareUDF($"in.curr_title", $"out.prev_title"))
	.withColumn("multiplication_factor", $"out_clicks" / $"in_clicks")
	.select($"in.curr_title", $"in_clicks", $"out_clicks", $"multiplication_factor")

display(joinedDF.orderBy($"multiplication_factor".desc))

In [ ]:
// Define a UDF for comparing article titles
val formatUDF = udf((title1: String) => title1.toLowerCase)

In [ ]:
// Fixed performance issue
val joinedDF = outClicksDF
	.join(inClicksDF, formatUDF($"in.curr_title") === formatUDF($"out.prev_title"))
	.withColumn("multiplication_factor", $"out_clicks" / $"in_clicks")
	.select($"in.curr_title", $"in_clicks", $"out_clicks", $"multiplication_factor")

display(joinedDF.orderBy($"multiplication_factor".desc))

In [18]:
// Fixed performance issue
val joinedDF2 = outClicksDF
	.join(inClicksDF, formatUDF($"in.curr_title") === formatUDF($"out.prev_title"))
	.withColumn("multiplication_factor", $"out_clicks" / $"in_clicks")
	.select($"in.curr_title", $"in_clicks", $"out_clicks", $"multiplication_factor")

display(joinedDF2.orderBy($"multiplication_factor"))

Name: Compile Error
Message: <console>:30: error: not found: value outClicksDF
       val joinedDF2 = outClicksDF
                       ^
StackTrace: 

 Interesting, looks like mostly cities, tech and colors.